The code for the sepCNN architecture was taken and adapted from this repository: https://github.com/google/eng-edu/tree/main/ml/guides/text_classification, according to this guide: https://developers.google.com/machine-learning/guides/text-classification

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence


# Vectorization parameters
# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Limit on the length of text sequences. Sequences longer than this
# will be truncated. 1 Lyric is longer than this in our case.
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, val_texts):
    """Vectorizes texts as sequence vectors.

    1 text = 1 sequence vector with fixed length.

    # Arguments
        train_texts: list, training text strings.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val, word_index: vectorized training and validation
            texts and word index dictionary.
    """
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index


In [2]:
def _get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.

    # Arguments
        num_classes: int, number of classes.

    # Returns
        units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation


In [3]:
from tensorflow.keras import models
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SeparableConv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D

def sepcnn_model(blocks,
                 filters,
                 kernel_size,
                 embedding_dim,
                 dropout_rate,
                 pool_size,
                 input_shape,
                 num_classes,
                 num_features,
                 use_pretrained_embedding=False,
                 is_embedding_trainable=False,
                 embedding_matrix=None):
    """Creates an instance of a separable CNN model.

    # Arguments
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        filters: int, output dimension of the layers.
        kernel_size: int, length of the convolution window.
        embedding_dim: int, dimension of the embedding vectors.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        pool_size: int, factor by which to downscale input at MaxPooling layer.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.
        num_features: int, number of words (embedding input dimension).
        use_pretrained_embedding: bool, true if pre-trained embedding is on.
        is_embedding_trainable: bool, true if embedding layer is trainable.
        embedding_matrix: dict, dictionary with embedding coefficients.

    # Returns
        A sepCNN model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()

    # Add embedding layer. If pre-trained embedding is used add weights to the
    # embeddings layer and set trainable to input is_embedding_trainable flag.
    if use_pretrained_embedding:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0],
                            weights=[embedding_matrix],
                            trainable=is_embedding_trainable))
    else:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0]))

    for _ in range(blocks-1):
        model.add(Dropout(rate=dropout_rate))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(MaxPooling1D(pool_size=pool_size))

    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(op_units, activation=op_activation))
    return model


In [8]:
from scikeras.wrappers import KerasClassifier
from load_data import load_data_album, load_data_singer, load_data_era
from sklearn.model_selection import cross_validate, GridSearchCV
import numpy as np
from tensorflow.keras.optimizers import Adam


def sepCNN_classifier(x_train, num_classes, num_features, use_pretrained_embedding=False, is_embedding_trainable=False, embedding_matrix=None):
    # fixed parameters
    blocks=2
    filters=64
    embedding_dim=300
    kernel_size=3
    pool_size=3

    # manually fine-tuned parameters
    epochs=200 
    learning_rate=0.015
    dropout_rate=0.4

    model = sepcnn_model(blocks=blocks,
                                        filters=filters,
                                        kernel_size=kernel_size,
                                        embedding_dim=embedding_dim,
                                        dropout_rate=dropout_rate,
                                        pool_size=pool_size,
                                        input_shape=x_train.shape[1:],
                                        num_classes=num_classes,
                                        num_features=num_features,
                                        use_pretrained_embedding=use_pretrained_embedding,
                                        is_embedding_trainable=is_embedding_trainable,
                                        embedding_matrix=embedding_matrix)

    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    # use scikeras to return the final model as a scikit-learn classifier
    clf = KerasClassifier(
        model,
        loss=loss,
        epochs = epochs
    )
    return clf

### SepCNN with learnable embeddings for classifying albums


In [9]:
# Load data and prepare
train_data, test_data = load_data_album()
train_texts = train_data["lyrics"]
train_labels = train_data["album_id"] - 1

# we load the test data as validation data here as it is necessary for the sequence_vectorize function, 
# but it is never used or accessed as we use cross validation
val_texts = test_data["lyrics"]
val_labels = test_data["album_id"] - 1

x_train, x_val, word_index = sequence_vectorize(
        train_texts, val_texts)

num_classes = len(np.unique(train_labels))
num_features = min(len(word_index) + 1, TOP_K)

# build the model 
clf = sepCNN_classifier(x_train, num_classes, num_features)

# use cross validation to find the accuracy of our model
scoring = ['precision_macro', 'recall_macro', "r2", "accuracy", "f1_macro"]
scores_cnn = cross_validate(clf, x_train, train_labels, scoring=scoring)
print(np.average(scores_cnn["test_accuracy"]))

INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp07ulyijg\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp07ulyijg\assets


Epoch 1/200
2/2 [==============================] - 1s 55ms/step - loss: 2.2074 - acc: 0.0000e+00
Epoch 2/200
2/2 [==============================] - 0s 49ms/step - loss: 2.2008 - acc: 0.0000e+00
Epoch 3/200
2/2 [==============================] - 0s 54ms/step - loss: 2.1877 - acc: 0.0000e+00
Epoch 4/200
2/2 [==============================] - 0s 53ms/step - loss: 2.1832 - acc: 0.0000e+00
Epoch 5/200
2/2 [==============================] - 0s 52ms/step - loss: 2.1894 - acc: 0.0000e+00
Epoch 6/200
2/2 [==============================] - 0s 50ms/step - loss: 2.1962 - acc: 0.0000e+00
Epoch 7/200
2/2 [==============================] - 0s 63ms/step - loss: 2.1991 - acc: 0.0000e+00
Epoch 8/200
2/2 [==============================] - 0s 53ms/step - loss: 2.1833 - acc: 0.0000e+00
Epoch 9/200
2/2 [==============================] - 0s 52ms/step - loss: 2.1956 - acc: 0.0000e+00
Epoch 10/200
2/2 [==============================] - 0s 59ms/step - loss: 2.1803 - acc: 0.0000e+00
Epoch 11/200
2/2 [===========

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmppexv520r\assets


...

INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp9finm0tw\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp9finm0tw\assets


Epoch 1/200
2/2 [==============================] - 1s 48ms/step - loss: 2.2054 - acc: 0.0167
Epoch 2/200
2/2 [==============================] - 0s 55ms/step - loss: 2.1940 - acc: 0.0000e+00
Epoch 3/200
2/2 [==============================] - 0s 54ms/step - loss: 2.1979 - acc: 0.0000e+00
Epoch 4/200
2/2 [==============================] - 0s 55ms/step - loss: 2.1972 - acc: 0.0000e+00
Epoch 5/200
2/2 [==============================] - 0s 54ms/step - loss: 2.1988 - acc: 0.0000e+00
Epoch 6/200
2/2 [==============================] - 0s 59ms/step - loss: 2.1854 - acc: 0.0000e+00
Epoch 7/200
2/2 [==============================] - 0s 46ms/step - loss: 2.1981 - acc: 0.0000e+00
Epoch 8/200
2/2 [==============================] - 0s 54ms/step - loss: 2.1996 - acc: 0.0000e+00
Epoch 9/200
2/2 [==============================] - 0s 55ms/step - loss: 2.1960 - acc: 0.0000e+00
Epoch 10/200
2/2 [==============================] - 0s 54ms/step - loss: 2.1944 - acc: 0.0000e+00
Epoch 11/200
2/2 [===============

1/1 [==============================] - 0s 96ms/step
0.13333333333333333


C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SepCNN with learnable embeddings for classifying eras


In [15]:
# SepCNN learnable embeddings Era
train_data, test_data = load_data_era()
train_texts = train_data["lyrics"]
train_labels = train_data["era_id"] - 1
val_texts = test_data["lyrics"]
val_labels = test_data["era_id"] - 1

x_train, x_val, word_index = sequence_vectorize(
        train_texts, val_texts)

num_classes = len(np.unique(train_labels))
num_features = min(len(word_index) + 1, TOP_K)

clf = sepCNN_classifier(x_train, num_classes, num_features)

scoring = ['precision_macro', 'recall_macro', "r2", "accuracy", "f1_macro"]
scores_cnn = cross_validate(clf, x_train, train_labels, scoring=scoring)
print(np.average(scores_cnn["test_accuracy"]))

INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp3h872u14\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp3h872u14\assets


Epoch 1/200
2/2 [==============================] - 1s 42ms/step - loss: 0.6969 - acc: 0.4333
Epoch 2/200
2/2 [==============================] - 0s 47ms/step - loss: 0.6894 - acc: 0.5500
Epoch 3/200
2/2 [==============================] - 0s 48ms/step - loss: 0.6856 - acc: 0.5500
Epoch 4/200
2/2 [==============================] - 0s 49ms/step - loss: 0.6916 - acc: 0.5500
Epoch 5/200
2/2 [==============================] - 0s 40ms/step - loss: 0.6897 - acc: 0.5500
Epoch 6/200
2/2 [==============================] - 0s 44ms/step - loss: 0.6947 - acc: 0.5500
Epoch 7/200
2/2 [==============================] - 0s 42ms/step - loss: 0.6847 - acc: 0.5500
Epoch 8/200
2/2 [==============================] - 0s 42ms/step - loss: 0.6879 - acc: 0.5500
Epoch 9/200
2/2 [==============================] - 0s 43ms/step - loss: 0.6871 - acc: 0.5500
Epoch 10/200
2/2 [==============================] - 0s 45ms/step - loss: 0.6881 - acc: 0.5500
Epoch 11/200
2/2 [==============================] - 0s 42ms/step - lo

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpyb6nzojj\assets


...

INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpy3927i6g\assets


Epoch 1/200
2/2 [==============================] - 1s 40ms/step - loss: 0.6939 - acc: 0.4500
Epoch 2/200
2/2 [==============================] - 0s 44ms/step - loss: 0.6902 - acc: 0.5667
Epoch 3/200
2/2 [==============================] - 0s 45ms/step - loss: 0.6856 - acc: 0.5667
Epoch 4/200
2/2 [==============================] - 0s 45ms/step - loss: 0.6859 - acc: 0.5667
Epoch 5/200
2/2 [==============================] - 0s 44ms/step - loss: 0.6805 - acc: 0.5667
Epoch 6/200
2/2 [==============================] - 0s 45ms/step - loss: 0.6874 - acc: 0.5667
Epoch 7/200
2/2 [==============================] - 0s 43ms/step - loss: 0.6889 - acc: 0.5667
Epoch 8/200
2/2 [==============================] - 0s 45ms/step - loss: 0.6847 - acc: 0.5667
Epoch 9/200
2/2 [==============================] - 0s 44ms/step - loss: 0.6928 - acc: 0.5667
Epoch 10/200
2/2 [==============================] - 0s 45ms/step - loss: 0.6839 - acc: 0.5667
Epoch 11/200
2/2 [==============================] - 0s 45ms/step - lo

INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp3a7hwze5\assets


Epoch 1/200
2/2 [==============================] - 1s 43ms/step - loss: 0.6958 - acc: 0.4333
Epoch 2/200
2/2 [==============================] - 0s 43ms/step - loss: 0.6894 - acc: 0.5667
Epoch 3/200
2/2 [==============================] - 0s 43ms/step - loss: 0.6844 - acc: 0.5667
Epoch 4/200
2/2 [==============================] - 0s 40ms/step - loss: 0.6859 - acc: 0.5667
Epoch 5/200
2/2 [==============================] - 0s 41ms/step - loss: 0.6879 - acc: 0.5667
Epoch 6/200
2/2 [==============================] - 0s 47ms/step - loss: 0.6818 - acc: 0.5667
Epoch 7/200
2/2 [==============================] - 0s 47ms/step - loss: 0.6847 - acc: 0.5667
Epoch 8/200
2/2 [==============================] - 0s 33ms/step - loss: 0.6850 - acc: 0.5667
Epoch 9/200
2/2 [==============================] - 0s 38ms/step - loss: 0.6832 - acc: 0.5667
Epoch 10/200
2/2 [==============================] - 0s 46ms/step - loss: 0.6861 - acc: 0.5667
Epoch 11/200
2/2 [==============================] - 0s 45ms/step - lo

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SepCNN with learnable embeddings for classifying singers


In [17]:
# SepCNN learnable embeddings Singer
train_data, test_data = load_data_singer()
train_texts = train_data["lyrics"]
train_labels = train_data["singer_id"] - 1
val_texts = test_data["lyrics"]
val_labels = test_data["singer_id"] - 1

x_train, x_val, word_index = sequence_vectorize(
        train_texts, val_texts)

num_classes = len(np.unique(train_labels))
num_features = min(len(word_index) + 1, TOP_K)

clf = sepCNN_classifier(x_train, num_classes, num_features)

scoring = ['precision_macro', 'recall_macro', "r2", "accuracy", "f1_macro"]
scores_cnn = cross_validate(clf, x_train, train_labels, scoring=scoring)
print(np.average(scores_cnn["test_accuracy"]))
print(np.average(scores_cnn["test_f1_macro"]))

INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpiod3m4pj\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpiod3m4pj\assets


Epoch 1/200
2/2 [==============================] - 1s 39ms/step - loss: 1.0814 - acc: 0.6500
Epoch 2/200
2/2 [==============================] - 0s 34ms/step - loss: 1.0155 - acc: 1.0000
Epoch 3/200
2/2 [==============================] - 0s 46ms/step - loss: 0.9739 - acc: 1.0000
Epoch 4/200
2/2 [==============================] - 0s 45ms/step - loss: 1.0059 - acc: 1.0000
Epoch 5/200
2/2 [==============================] - 0s 41ms/step - loss: 0.9900 - acc: 1.0000
Epoch 6/200
2/2 [==============================] - 0s 46ms/step - loss: 0.9852 - acc: 1.0000
Epoch 7/200
2/2 [==============================] - 0s 42ms/step - loss: 0.9702 - acc: 1.0000
Epoch 8/200
2/2 [==============================] - 0s 45ms/step - loss: 1.0081 - acc: 1.0000
Epoch 9/200
2/2 [==============================] - 0s 42ms/step - loss: 0.9981 - acc: 1.0000
Epoch 10/200
2/2 [==============================] - 0s 42ms/step - loss: 0.9773 - acc: 1.0000
Epoch 11/200
2/2 [==============================] - 0s 41ms/step - lo

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpcb9rgw8y\assets


...

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpevg52x4j\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpevg52x4j\assets


Epoch 1/200
2/2 [==============================] - 1s 43ms/step - loss: 1.0700 - acc: 0.7000
Epoch 2/200
2/2 [==============================] - 0s 40ms/step - loss: 0.9972 - acc: 1.0000
Epoch 3/200
2/2 [==============================] - 0s 46ms/step - loss: 0.9692 - acc: 1.0000
Epoch 4/200
2/2 [==============================] - 0s 46ms/step - loss: 1.0104 - acc: 1.0000
Epoch 5/200
2/2 [==============================] - 0s 43ms/step - loss: 0.9862 - acc: 1.0000
Epoch 6/200
2/2 [==============================] - 0s 42ms/step - loss: 0.9697 - acc: 1.0000
Epoch 7/200
2/2 [==============================] - 0s 39ms/step - loss: 0.9596 - acc: 1.0000
Epoch 8/200
2/2 [==============================] - 0s 43ms/step - loss: 0.9697 - acc: 1.0000
Epoch 9/200
2/2 [==============================] - 0s 42ms/step - loss: 0.9649 - acc: 1.0000
Epoch 10/200
2/2 [==============================] - 0s 42ms/step - loss: 0.9820 - acc: 1.0000
Epoch 11/200
2/2 [==============================] - 0s 45ms/step - lo

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Pretrained word embeddings

For pretrained word embeddings we use GloVe. As the embedding file we use in this project (glove.6B.300d.txt) is too big for uploading. If you want to run the project please download the corresponding embeddings from Glove: https://nlp.stanford.edu/projects/glove/ and place the .txt file in the project repository.

In [11]:
def _get_embedding_matrix(word_index, embedding_dim):
    """Gets embedding matrix from the embedding index data.

    # Arguments
        word_index: dict, word to index map that was generated from the data.
        embedding_data_dir: string, path to the pre-training embeddings.
        embedding_dim: int, dimension of the embedding vectors.

    # Returns
        dict, word vectors for words in word_index from pre-trained embedding.

    # References:
        https://nlp.stanford.edu/projects/glove/

        Download and uncompress archive from:
        http://nlp.stanford.edu/data/glove.6B.zip
    """

    # Read the pre-trained embedding file and get word to word vector mappings.
    embedding_matrix_all = {}

    # We are using 200d GloVe embeddings.
    fname = 'glove.6B.300d.txt'
    with open(fname) as f:
        for line in f:  # Every line contains word followed by the vector value
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embedding_matrix_all[word] = coefs

    # Prepare embedding matrix with just the words in our word_index dictionary
    num_words = min(len(word_index) + 1, TOP_K)
    embedding_matrix = np.zeros((num_words, embedding_dim))

    for word, i in word_index.items():
        if i >= TOP_K:
            continue
        embedding_vector = embedding_matrix_all.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

### SepCNN with pretrained embeddings for classifying albums

In [12]:
# SepCNN pretrained embeddings Album
train_data, test_data = load_data_album()
train_texts = train_data["lyrics"]
train_labels = train_data["album_id"] - 1
val_texts = test_data["lyrics"]
val_labels = test_data["album_id"] - 1

x_train, x_val, word_index = sequence_vectorize(
        train_texts, val_texts)

num_classes = len(np.unique(train_labels))
num_features = min(len(word_index) + 1, TOP_K)

embedding_matrix = _get_embedding_matrix(
        word_index, 300)

clf = sepCNN_classifier(x_train, num_classes, num_features, use_pretrained_embedding=True, is_embedding_trainable=True, embedding_matrix=embedding_matrix)

scores_cnn = cross_validate(clf, x_train, train_labels, scoring=scoring)
print(np.average(scores_cnn["test_accuracy"]))

INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp_l_gf2pm\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp_l_gf2pm\assets


Epoch 1/200
2/2 [==============================] - 1s 54ms/step - loss: 2.2076 - acc: 0.0000e+00
Epoch 2/200
2/2 [==============================] - 0s 49ms/step - loss: 2.1920 - acc: 0.0000e+00
Epoch 3/200
2/2 [==============================] - 0s 52ms/step - loss: 2.1935 - acc: 0.0000e+00
Epoch 4/200
2/2 [==============================] - 0s 54ms/step - loss: 2.1924 - acc: 0.0000e+00
Epoch 5/200
2/2 [==============================] - 0s 51ms/step - loss: 2.1901 - acc: 0.0000e+00
Epoch 6/200
2/2 [==============================] - 0s 50ms/step - loss: 2.1867 - acc: 0.0000e+00
Epoch 7/200
2/2 [==============================] - 0s 52ms/step - loss: 2.1805 - acc: 0.0000e+00
Epoch 8/200
2/2 [==============================] - 0s 57ms/step - loss: 2.1803 - acc: 0.0000e+00
Epoch 9/200
2/2 [==============================] - 0s 61ms/step - loss: 2.1989 - acc: 0.0000e+00
Epoch 10/200
2/2 [==============================] - 0s 55ms/step - loss: 2.1733 - acc: 0.0000e+00
Epoch 11/200
2/2 [===========

1/1 [==============================] - 0s 96ms/step


C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpttgq16sl\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpttgq16sl\assets


Epoch 1/200
2/2 [==============================] - 1s 47ms/step - loss: 2.2035 - acc: 0.0000e+00
Epoch 2/200
2/2 [==============================] - 0s 52ms/step - loss: 2.2017 - acc: 0.0000e+00
Epoch 3/200
2/2 [==============================] - 0s 53ms/step - loss: 2.1935 - acc: 0.0000e+00
Epoch 4/200
2/2 [==============================] - 0s 55ms/step - loss: 2.1938 - acc: 0.0000e+00
Epoch 5/200
2/2 [==============================] - 0s 50ms/step - loss: 2.1945 - acc: 0.0000e+00
Epoch 6/200
2/2 [==============================] - 0s 63ms/step - loss: 2.1860 - acc: 0.0000e+00
Epoch 7/200
2/2 [==============================] - 0s 49ms/step - loss: 2.1886 - acc: 0.0000e+00
Epoch 8/200
2/2 [==============================] - 0s 56ms/step - loss: 2.1849 - acc: 0.0000e+00
Epoch 9/200
2/2 [==============================] - 0s 58ms/step - loss: 2.2077 - acc: 0.0000e+00
Epoch 10/200
2/2 [==============================] - 0s 59ms/step - loss: 2.2045 - acc: 0.0000e+00
Epoch 11/200
2/2 [===========

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SepCNN with pretrained embeddings for classifying eras

In [13]:
# SepCNN pretrained embeddings Era
train_data, test_data = load_data_era()
train_texts = train_data["lyrics"]
train_labels = train_data["era_id"] - 1
val_texts = test_data["lyrics"]
val_labels = test_data["era_id"] - 1

x_train, x_val, word_index = sequence_vectorize(
        train_texts, val_texts)

num_classes = len(np.unique(train_labels))
num_features = min(len(word_index) + 1, TOP_K)

embedding_matrix = _get_embedding_matrix(
        word_index, 300)

clf = sepCNN_classifier(x_train, num_classes, num_features, use_pretrained_embedding=True, is_embedding_trainable=True, embedding_matrix=embedding_matrix)

scores_cnn = cross_validate(clf, x_train, train_labels, scoring=scoring)
print(np.average(scores_cnn["test_accuracy"]))

INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpkqxgjp97\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpkqxgjp97\assets


Epoch 1/200
2/2 [==============================] - 1s 49ms/step - loss: 0.6926 - acc: 0.4833
Epoch 2/200
2/2 [==============================] - 0s 55ms/step - loss: 0.6898 - acc: 0.5500
Epoch 3/200
2/2 [==============================] - 0s 47ms/step - loss: 0.6890 - acc: 0.5500
Epoch 4/200
2/2 [==============================] - 0s 54ms/step - loss: 0.6939 - acc: 0.5500
Epoch 5/200
2/2 [==============================] - 0s 47ms/step - loss: 0.6839 - acc: 0.5500
Epoch 6/200
2/2 [==============================] - 0s 39ms/step - loss: 0.6930 - acc: 0.5500
Epoch 7/200
2/2 [==============================] - 0s 47ms/step - loss: 0.6951 - acc: 0.5500
Epoch 8/200
2/2 [==============================] - 0s 50ms/step - loss: 0.6831 - acc: 0.5500
Epoch 9/200
2/2 [==============================] - 0s 49ms/step - loss: 0.6874 - acc: 0.5500
Epoch 10/200
2/2 [==============================] - 0s 41ms/step - loss: 0.6887 - acc: 0.5333
Epoch 11/200
2/2 [==============================] - 0s 40ms/step - lo

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp04fp7wnt\assets


...

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpw8i1r2ff\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpw8i1r2ff\assets


Epoch 1/200
2/2 [==============================] - 1s 59ms/step - loss: 0.6919 - acc: 0.5333
Epoch 2/200
2/2 [==============================] - 0s 58ms/step - loss: 0.6856 - acc: 0.5667
Epoch 3/200
2/2 [==============================] - 0s 65ms/step - loss: 0.6916 - acc: 0.5667
Epoch 4/200
2/2 [==============================] - 0s 64ms/step - loss: 0.6940 - acc: 0.5667
Epoch 5/200
2/2 [==============================] - 0s 59ms/step - loss: 0.6869 - acc: 0.5667
Epoch 6/200
2/2 [==============================] - 0s 50ms/step - loss: 0.6831 - acc: 0.5667
Epoch 7/200
2/2 [==============================] - 0s 60ms/step - loss: 0.6814 - acc: 0.5667
Epoch 8/200
2/2 [==============================] - 0s 50ms/step - loss: 0.6920 - acc: 0.5667
Epoch 9/200
2/2 [==============================] - 0s 68ms/step - loss: 0.6881 - acc: 0.5667
Epoch 10/200
2/2 [==============================] - 0s 52ms/step - loss: 0.6793 - acc: 0.5667
Epoch 11/200
2/2 [==============================] - 0s 63ms/step - lo

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SepCNN with pretrained embeddings for classifying singers

In [26]:
# SepCNN pretrained embeddings Singer
train_data, test_data = load_data_singer()
train_texts = train_data["lyrics"]
train_labels = train_data["singer_id"] - 1
val_texts = test_data["lyrics"]
val_labels = test_data["singer_id"] - 1

x_train, x_val, word_index = sequence_vectorize(
        train_texts, val_texts)

num_classes = len(np.unique(train_labels))
num_features = min(len(word_index) + 1, TOP_K)

embedding_matrix = _get_embedding_matrix(
        word_index, 300)

clf = sepCNN_classifier(x_train, num_classes, num_features, use_pretrained_embedding=True, is_embedding_trainable=True, embedding_matrix=embedding_matrix)

scores_cnn = cross_validate(clf, x_train, train_labels, scoring=scoring)
print(np.average(scores_cnn["test_accuracy"]))
print(np.average(scores_cnn["test_f1_macro"]))

INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp5h8oy16t\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp5h8oy16t\assets


Epoch 1/200
2/2 [==============================] - 1s 40ms/step - loss: 1.0877 - acc: 0.5167
Epoch 2/200
2/2 [==============================] - 0s 45ms/step - loss: 1.0287 - acc: 1.0000
Epoch 3/200
2/2 [==============================] - 0s 38ms/step - loss: 0.9934 - acc: 1.0000
Epoch 4/200
2/2 [==============================] - 0s 42ms/step - loss: 1.0161 - acc: 1.0000
Epoch 5/200
2/2 [==============================] - 0s 39ms/step - loss: 1.0007 - acc: 1.0000
Epoch 6/200
2/2 [==============================] - 0s 50ms/step - loss: 0.9778 - acc: 1.0000
Epoch 7/200
2/2 [==============================] - 0s 46ms/step - loss: 0.9845 - acc: 1.0000
Epoch 8/200
2/2 [==============================] - 0s 50ms/step - loss: 0.9831 - acc: 1.0000
Epoch 9/200
2/2 [==============================] - 0s 42ms/step - loss: 0.9739 - acc: 1.0000
Epoch 10/200
2/2 [==============================] - 0s 47ms/step - loss: 0.9904 - acc: 1.0000
Epoch 11/200
2/2 [==============================] - 0s 55ms/step - lo

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmp6gkwlv33\assets


...

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpe_655oos\assets


INFO:tensorflow:Assets written to: C:\Users\nikos\AppData\Local\Temp\tmpe_655oos\assets


Epoch 1/200
2/2 [==============================] - 1s 47ms/step - loss: 1.0768 - acc: 0.6167
Epoch 2/200
2/2 [==============================] - 0s 47ms/step - loss: 1.0108 - acc: 1.0000
Epoch 3/200
2/2 [==============================] - 0s 46ms/step - loss: 0.9935 - acc: 1.0000
Epoch 4/200
2/2 [==============================] - 0s 44ms/step - loss: 0.9968 - acc: 1.0000
Epoch 5/200
2/2 [==============================] - 0s 51ms/step - loss: 0.9836 - acc: 1.0000
Epoch 6/200
2/2 [==============================] - 0s 47ms/step - loss: 0.9764 - acc: 1.0000
Epoch 7/200
2/2 [==============================] - 0s 47ms/step - loss: 0.9924 - acc: 1.0000
Epoch 8/200
2/2 [==============================] - 0s 62ms/step - loss: 0.9767 - acc: 1.0000
Epoch 9/200
2/2 [==============================] - 0s 49ms/step - loss: 0.9725 - acc: 1.0000
Epoch 10/200
2/2 [==============================] - 0s 48ms/step - loss: 0.9716 - acc: 1.0000
Epoch 11/200
2/2 [==============================] - 0s 47ms/step - lo

C:\Users\nikos\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Legacy Code

not used in project, only relevant for reference

In [ ]:
"""Module to train sequence model.

Vectorizes training and validation texts into sequences and uses that for
training a sequence model - a sepCNN model. We use sequence model for text
classification when the ratio of number of samples to number of words per
sample for the given dataset is very large (>~15K).
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import time

from tensorflow.keras.optimizers import Adam

import tensorflow as tf
import numpy as np

from load_data import load_data_album, load_data_singer, load_data_era

FLAGS = None

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000


def train_sequence_model_singer(learning_rate=1e-3,
                         epochs=1000,
                         batch_size=128,
                         blocks=2,
                         filters=64,
                         dropout_rate=0.2,
                         embedding_dim=200,
                         kernel_size=3,
                         pool_size=3):
    """Trains sequence model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        filters: int, output dimension of sepCNN layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.
        embedding_dim: int, dimension of the embedding vectors.
        kernel_size: int, length of the convolution window.
        pool_size: int, factor by which to downscale input at MaxPooling layer.

    # Raises
        ValueError: If validation data has label values which were not seen
            in the training data.
    """
    # Get the data.
    train_data, test_data = load_data_singer()
    train_texts = train_data["lyrics"]
    train_labels = train_data["singer_id"] - 1
    val_texts = test_data["lyrics"]
    val_labels = test_data["singer_id"] - 1

    # Verify that validation labels are in the same range as training labels.
    num_classes = len(np.unique(train_labels))
    unexpected_labels = [v for v in val_labels if v not in range(num_classes)]
    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the '
                         'labels in the validation set are in the same range '
                         'as training labels.'.format(
                             unexpected_labels=unexpected_labels))

    # Vectorize texts.
    x_train, x_val, word_index = sequence_vectorize(
            train_texts, val_texts)

    # Number of features will be the embedding input dimension. Add 1 for the
    # reserved index 0.
    num_features = min(len(word_index) + 1, TOP_K)

    # Create model instance.
    model = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features)

    # Compile model with learning parameters.
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=500)]

    # Train and validate model.
    history = model.fit(
            x_train,
            train_labels,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=(x_val, val_labels),
            verbose=2,  # Logs once per epoch.
            batch_size=batch_size)

    # Print results.
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    return history['val_acc'][-1], history['val_loss'][-1]

def train_sequence_model_era(learning_rate=1e-3,
                         epochs=1000,
                         batch_size=128,
                         blocks=2,
                         filters=64,
                         dropout_rate=0.2,
                         embedding_dim=200,
                         kernel_size=3,
                         pool_size=3):
    """Trains sequence model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        filters: int, output dimension of sepCNN layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.
        embedding_dim: int, dimension of the embedding vectors.
        kernel_size: int, length of the convolution window.
        pool_size: int, factor by which to downscale input at MaxPooling layer.

    # Raises
        ValueError: If validation data has label values which were not seen
            in the training data.
    """
    # Get the data.
    train_data, test_data = load_data_era()
    train_texts = train_data["lyrics"]
    train_labels = train_data["era_id"] - 1
    val_texts = test_data["lyrics"]
    val_labels = test_data["era_id"] - 1

    # Verify that validation labels are in the same range as training labels.
    num_classes = len(np.unique(train_labels))
    unexpected_labels = [v for v in val_labels if v not in range(num_classes)]
    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the '
                         'labels in the validation set are in the same range '
                         'as training labels.'.format(
                             unexpected_labels=unexpected_labels))

    # Vectorize texts.
    x_train, x_val, word_index = sequence_vectorize(
            train_texts, val_texts)

    # Number of features will be the embedding input dimension. Add 1 for the
    # reserved index 0.
    num_features = min(len(word_index) + 1, TOP_K)

    # Create model instance.
    model = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features)

    # Compile model with learning parameters.
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=500)]

    # Train and validate model.
    history = model.fit(
            x_train,
            train_labels,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=(x_val, val_labels),
            verbose=2,  # Logs once per epoch.
            batch_size=batch_size)

    # Print results.
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    return history['val_acc'][-1], history['val_loss'][-1]

def train_sequence_model_album(learning_rate=1e-3,
                         epochs=1000,
                         batch_size=128,
                         blocks=2,
                         filters=64,
                         dropout_rate=0.2,
                         embedding_dim=200,
                         kernel_size=3,
                         pool_size=3):
    """Trains sequence model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        filters: int, output dimension of sepCNN layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.
        embedding_dim: int, dimension of the embedding vectors.
        kernel_size: int, length of the convolution window.
        pool_size: int, factor by which to downscale input at MaxPooling layer.

    # Raises
        ValueError: If validation data has label values which were not seen
            in the training data.
    """
    # Get the data.
    train_data, test_data = load_data_album()
    train_texts = train_data["lyrics"]
    train_labels = train_data["album_id"] - 1
    val_texts = test_data["lyrics"]
    val_labels = test_data["album_id"] - 1

    # Verify that validation labels are in the same range as training labels.
    num_classes = len(np.unique(train_labels))
    unexpected_labels = [v for v in val_labels if v not in range(num_classes)]
    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the '
                         'labels in the validation set are in the same range '
                         'as training labels.'.format(
                             unexpected_labels=unexpected_labels))

    # Vectorize texts.
    x_train, x_val, word_index = sequence_vectorize(
            train_texts, val_texts)

    # Number of features will be the embedding input dimension. Add 1 for the
    # reserved index 0.
    num_features = min(len(word_index) + 1, TOP_K)

    # Create model instance.
    model = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features)

    # Compile model with learning parameters.
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=500)]

    # Train and validate model.
    history = model.fit(
            x_train,
            train_labels,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=(x_val, val_labels),
            verbose=2,  # Logs once per epoch.
            batch_size=batch_size)

    # Print results.
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    return history['val_acc'][-1], history['val_loss'][-1]

In [ ]:
train_sequence_model_album(epochs=300, learning_rate=0.015, dropout_rate=0.3)

In [ ]:
train_sequence_model_era(epochs=300, learning_rate=0.04, dropout_rate=0.4)

In [ ]:
train_sequence_model_singer(epochs=200, learning_rate=0.01, dropout_rate=0.4)

In [ ]:
def _get_embedding_matrix(word_index, embedding_dim):
    """Gets embedding matrix from the embedding index data.

    # Arguments
        word_index: dict, word to index map that was generated from the data.
        embedding_data_dir: string, path to the pre-training embeddings.
        embedding_dim: int, dimension of the embedding vectors.

    # Returns
        dict, word vectors for words in word_index from pre-trained embedding.

    # References:
        https://nlp.stanford.edu/projects/glove/

        Download and uncompress archive from:
        http://nlp.stanford.edu/data/glove.6B.zip
    """

    # Read the pre-trained embedding file and get word to word vector mappings.
    embedding_matrix_all = {}

    # We are using 200d GloVe embeddings.
    fname = 'glove.6B.300d.txt'
    with open(fname) as f:
        for line in f:  # Every line contains word followed by the vector value
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embedding_matrix_all[word] = coefs

    # Prepare embedding matrix with just the words in our word_index dictionary
    num_words = min(len(word_index) + 1, TOP_K)
    embedding_matrix = np.zeros((num_words, embedding_dim))

    for word, i in word_index.items():
        if i >= TOP_K:
            continue
        embedding_vector = embedding_matrix_all.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


def train_fine_tuned_sequence_model_album(learning_rate=1e-3,
                                    epochs=1000,
                                    batch_size=128,
                                    blocks=2,
                                    filters=64,
                                    dropout_rate=0.2,
                                    embedding_dim=300,
                                    kernel_size=3,
                                    pool_size=3):
    """Trains sequence model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
        embedding_data_dir: string, path to the pre-training embeddings.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        filters: int, output dimension of sepCNN layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.
        embedding_dim: int, dimension of the embedding vectors.
        kernel_size: int, length of the convolution window.
        pool_size: int, factor by which to downscale input at MaxPooling layer.

    # Raises
        ValueError: If validation data has label values which were not seen
            in the training data.
    """
    # Get the data.
    train_data, test_data = load_data_album()
    train_texts = train_data["lyrics"]
    train_labels = train_data["album_id"] - 1
    val_texts = test_data["lyrics"]
    val_labels = test_data["album_id"] - 1

    # Verify that validation labels are in the same range as training labels.
    num_classes = len(np.unique(train_labels))
    unexpected_labels = [v for v in val_labels if v not in range(num_classes)]
    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the '
                         'labels in the validation set are in the same range '
                         'as training labels.'.format(
                             unexpected_labels=unexpected_labels))

    # Vectorize texts.
    x_train, x_val, word_index = sequence_vectorize(
            train_texts, val_texts)

    # Number of features will be the embedding input dimension. Add 1 for the
    # reserved index 0.
    num_features = min(len(word_index) + 1, TOP_K)

    embedding_matrix = _get_embedding_matrix(
        word_index, embedding_dim)

    # Create model instance. First time we will train rest of network while
    # keeping embedding layer weights frozen. So, we set
    # is_embedding_trainable as False.
    model = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features,
                                     use_pretrained_embedding=True,
                                     is_embedding_trainable=False,
                                     embedding_matrix=embedding_matrix)

    # Compile model with learning parameters.
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=300)]

    # Train and validate model.
    model.fit(x_train,
              train_labels,
              epochs=epochs,
              callbacks=callbacks,
              validation_data=(x_val, val_labels),
              verbose=2,  # Logs once per epoch.
              batch_size=batch_size)

    # Save the model.
    model.save_weights('sequence_model_with_pre_trained_embedding.h5')

    # Create another model instance. This time we will unfreeze the embedding
    # layer and let it fine-tune to the given dataset.
    model2 = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features,
                                     use_pretrained_embedding=True,
                                     is_embedding_trainable=True,
                                     embedding_matrix=embedding_matrix)

    # Compile model with learning parameters.
    optimizer2 = Adam(learning_rate=learning_rate)
    model2.compile(optimizer=optimizer2, loss=loss, metrics=['acc'])

    # Load the weights that we had saved into this new model.
    model2.load_weights('sequence_model_with_pre_trained_embedding.h5')

    # Train and validate model.
    history = model2.fit(x_train,
                        train_labels,
                        epochs=epochs,
                        callbacks=callbacks,
                        validation_data=(x_val, val_labels),
                        verbose=2,  # Logs once per epoch.
                        batch_size=batch_size)

    # Print results.
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    # Save model.
    return history['val_acc'][-1], history['val_loss'][-1]

def train_fine_tuned_sequence_model_singer(learning_rate=1e-3,
                                    epochs=1000,
                                    batch_size=128,
                                    blocks=2,
                                    filters=64,
                                    dropout_rate=0.2,
                                    embedding_dim=300,
                                    kernel_size=3,
                                    pool_size=3):
    """Trains sequence model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
        embedding_data_dir: string, path to the pre-training embeddings.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        filters: int, output dimension of sepCNN layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.
        embedding_dim: int, dimension of the embedding vectors.
        kernel_size: int, length of the convolution window.
        pool_size: int, factor by which to downscale input at MaxPooling layer.

    # Raises
        ValueError: If validation data has label values which were not seen
            in the training data.
    """
    # Get the data.
    train_data, test_data = load_data_singer()
    train_texts = train_data["lyrics"]
    train_labels = train_data["singer_id"] - 1
    val_texts = test_data["lyrics"]
    val_labels = test_data["singer_id"] - 1

    # Verify that validation labels are in the same range as training labels.
    num_classes = len(np.unique(train_labels))
    unexpected_labels = [v for v in val_labels if v not in range(num_classes)]
    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the '
                         'labels in the validation set are in the same range '
                         'as training labels.'.format(
                             unexpected_labels=unexpected_labels))

    # Vectorize texts.
    x_train, x_val, word_index = sequence_vectorize(
            train_texts, val_texts)

    # Number of features will be the embedding input dimension. Add 1 for the
    # reserved index 0.
    num_features = min(len(word_index) + 1, TOP_K)

    embedding_matrix = _get_embedding_matrix(
        word_index, embedding_dim)

    # Create model instance. First time we will train rest of network while
    # keeping embedding layer weights frozen. So, we set
    # is_embedding_trainable as False.
    model = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features,
                                     use_pretrained_embedding=True,
                                     is_embedding_trainable=False,
                                     embedding_matrix=embedding_matrix)

    # Compile model with learning parameters.
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=300)]

    # Train and validate model.
    model.fit(x_train,
              train_labels,
              epochs=epochs,
              callbacks=callbacks,
              validation_data=(x_val, val_labels),
              verbose=2,  # Logs once per epoch.
              batch_size=batch_size)

    # Save the model.
    model.save_weights('sequence_model_with_pre_trained_embedding.h5')

    # Create another model instance. This time we will unfreeze the embedding
    # layer and let it fine-tune to the given dataset.
    model2 = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features,
                                     use_pretrained_embedding=True,
                                     is_embedding_trainable=True,
                                     embedding_matrix=embedding_matrix)

    # Compile model with learning parameters.
    optimizer2 = Adam(learning_rate=learning_rate)
    model2.compile(optimizer=optimizer2, loss=loss, metrics=['acc'])

    # Load the weights that we had saved into this new model.
    model2.load_weights('sequence_model_with_pre_trained_embedding.h5')

    # Train and validate model.
    history = model2.fit(x_train,
                        train_labels,
                        epochs=epochs,
                        callbacks=callbacks,
                        validation_data=(x_val, val_labels),
                        verbose=2,  # Logs once per epoch.
                        batch_size=batch_size)

    # Print results.
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    # Save model.
    return history['val_acc'][-1], history['val_loss'][-1]

def train_fine_tuned_sequence_model_era(learning_rate=1e-3,
                                    epochs=1000,
                                    batch_size=128,
                                    blocks=2,
                                    filters=64,
                                    dropout_rate=0.2,
                                    embedding_dim=300,
                                    kernel_size=3,
                                    pool_size=3):
    """Trains sequence model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
        embedding_data_dir: string, path to the pre-training embeddings.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        filters: int, output dimension of sepCNN layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.
        embedding_dim: int, dimension of the embedding vectors.
        kernel_size: int, length of the convolution window.
        pool_size: int, factor by which to downscale input at MaxPooling layer.

    # Raises
        ValueError: If validation data has label values which were not seen
            in the training data.
    """
    # Get the data.
    train_data, test_data = load_data_era()
    train_texts = train_data["lyrics"]
    train_labels = train_data["era_id"] - 1
    val_texts = test_data["lyrics"]
    val_labels = test_data["era_id"] - 1

    # Verify that validation labels are in the same range as training labels.
    num_classes = len(np.unique(train_labels))
    unexpected_labels = [v for v in val_labels if v not in range(num_classes)]
    if len(unexpected_labels):
        raise ValueError('Unexpected label values found in the validation set:'
                         ' {unexpected_labels}. Please make sure that the '
                         'labels in the validation set are in the same range '
                         'as training labels.'.format(
                             unexpected_labels=unexpected_labels))

    # Vectorize texts.
    x_train, x_val, word_index = sequence_vectorize(
            train_texts, val_texts)

    # Number of features will be the embedding input dimension. Add 1 for the
    # reserved index 0.
    num_features = min(len(word_index) + 1, TOP_K)

    embedding_matrix = _get_embedding_matrix(
        word_index, embedding_dim)

    # Create model instance. First time we will train rest of network while
    # keeping embedding layer weights frozen. So, we set
    # is_embedding_trainable as False.
    model = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features,
                                     use_pretrained_embedding=True,
                                     is_embedding_trainable=False,
                                     embedding_matrix=embedding_matrix)

    # Compile model with learning parameters.
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=300)]

    # Train and validate model.
    model.fit(x_train,
              train_labels,
              epochs=epochs,
              callbacks=callbacks,
              validation_data=(x_val, val_labels),
              verbose=2,  # Logs once per epoch.
              batch_size=batch_size)

    # Save the model.
    model.save_weights('sequence_model_with_pre_trained_embedding.h5')

    # Create another model instance. This time we will unfreeze the embedding
    # layer and let it fine-tune to the given dataset.
    model2 = sepcnn_model(blocks=blocks,
                                     filters=filters,
                                     kernel_size=kernel_size,
                                     embedding_dim=embedding_dim,
                                     dropout_rate=dropout_rate,
                                     pool_size=pool_size,
                                     input_shape=x_train.shape[1:],
                                     num_classes=num_classes,
                                     num_features=num_features,
                                     use_pretrained_embedding=True,
                                     is_embedding_trainable=True,
                                     embedding_matrix=embedding_matrix)

    # Compile model with learning parameters.
    optimizer2 = Adam(learning_rate=learning_rate)
    model2.compile(optimizer=optimizer2, loss=loss, metrics=['acc'])

    # Load the weights that we had saved into this new model.
    model2.load_weights('sequence_model_with_pre_trained_embedding.h5')

    # Train and validate model.
    history = model2.fit(x_train,
                        train_labels,
                        epochs=epochs,
                        callbacks=callbacks,
                        validation_data=(x_val, val_labels),
                        verbose=2,  # Logs once per epoch.
                        batch_size=batch_size)

    # Print results.
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    # Save model.
    return history['val_acc'][-1], history['val_loss'][-1]

In [ ]:
train_fine_tuned_sequence_model_album(dropout_rate=0.3, epochs=200, learning_rate=0.015)

In [ ]:
train_fine_tuned_sequence_model_era(dropout_rate=0.4, epochs=150, learning_rate=0.01)

In [ ]:
train_fine_tuned_sequence_model_singer(dropout_rate=0.3, epochs=150, learning_rate=0.01)